In [1]:
# Install dependencies
!pip install transformers datasets accelerate -q

# Import libraries
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments



In [2]:
# 1. Load dataset
dataset = load_dataset("imdb")
train_dataset = dataset["train"].shuffle(seed=42).select(range(4000))  # small subset for quick training
test_dataset = dataset["test"].shuffle(seed=42).select(range(1000))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
# 2. Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# 3. Tokenize data
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])



Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
# 4. Load model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# 5. Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False,
    report_to="none" # Disable wandb logging
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# 6. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)



In [14]:
# 7. Train model
trainer.train()



Step,Training Loss
10,0.696300
20,0.663700
30,0.584500
40,0.454200
50,0.393800
60,0.409300
70,0.339600
80,0.322900
90,0.214000
100,0.391800


TrainOutput(global_step=500, training_loss=0.30088457345962527, metrics={'train_runtime': 183.413, 'train_samples_per_second': 43.617, 'train_steps_per_second': 2.726, 'total_flos': 529869594624000.0, 'train_loss': 0.30088457345962527, 'epoch': 2.0})

In [16]:
# 8. Evaluate
results = trainer.evaluate()
print("Evaluation results:", results)



Evaluation results: {'eval_loss': 0.30068323016166687, 'eval_runtime': 6.6538, 'eval_samples_per_second': 150.29, 'eval_steps_per_second': 9.468, 'epoch': 2.0}


In [28]:
# 9. Test on new text
from transformers import logging
logging.set_verbosity_error()

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    # Move inputs to the same device as the model
    inputs = {key: val.to(model.device) for key, val in inputs.items()}
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, dim=1)
    return "Positive 😀" if preds.item() == 1 else "Negative 😡"

print(predict("This movie was fantastic!"))
print(predict("I hated every moment of this movie."))

Positive 😀
Negative 😡


In [27]:
# turn off log
from transformers import logging
logging.set_verbosity_error()


# Function to evaluate a model on the test dataset
def evaluate_model_on_testset(model, dataset):
    correct_predictions = 0
    total_predictions = 0

    # Ensure the model is in evaluation mode and on the correct device
    model.eval()
    device = model.device

    with torch.no_grad():
        for i in range(len(dataset)):
            item = dataset[i]
            inputs = {
                "input_ids": item["input_ids"].unsqueeze(0).to(device),
                "attention_mask": item["attention_mask"].unsqueeze(0).to(device),
            }
            labels = item["label"].unsqueeze(0).to(device)

            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)

            correct_predictions += (preds == labels).sum().item()
            total_predictions += 1

    accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
    return accuracy

# Evaluate the fine-tuned model (loaded in cell -PTyabEUsU2g and trained in cell kaff8_mMsd-i)
fine_tuned_model_accuracy = evaluate_model_on_testset(model, test_dataset)
print(f"Accuracy of the fine-tuned model on the test set: {fine_tuned_model_accuracy:.2f}%")

# Evaluate the pre-trained model (instantiate a new one without training)
# Use the tokenizer loaded in cell zB1aSAX-sK6g and test_dataset from cell YZvQOwtgsQEl
pretrained_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
pretrained_model.to(model.device) # Move to the same device as the fine-tuned model

pretrained_model_accuracy = evaluate_model_on_testset(pretrained_model, test_dataset)
print(f"Accuracy of the pre-trained model on the test set: {pretrained_model_accuracy:.2f}%")

Accuracy of the fine-tuned model on the test set: 88.40%
Accuracy of the pre-trained model on the test set: 49.60%
